# Exercises

## Acquire (`acquire.py`)

### Zillow
**For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.**

**You will have a** 
`zillow.ipynb` 
**file and a helper file for each section in the pipeline.**

In [1]:
###############################################################################
### python imports                                                          ###
###############################################################################

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

# from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import acquire as acq
import prepare as prep

from debug import local_settings, timeifdebug, timeargsifdebug, frame_splain
from dfo import DFO, set_dfo

In [3]:
local_settings.splain=True
local_settings.debug=True
splain = local_settings.splain
debug = local_settings.debug

In [4]:
from wrangle_zillow import get_zillow_data, prep_zillow_data, sqls

#### Acquire & Summarize

1. **Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.**

   - *Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for* 
`airconditioningtypeid`
*.*
   - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
   - Only include properties that include a latitude and longitude value.

In [5]:
dfo = set_dfo(get_zillow_data(sql=sqls['mvp']), splain=True)

2019-11-06 10:51:08 starting get_zillow_data
2019-11-06 10:51:08 starting sql_df
2019-11-06 10:51:08 starting get_db_url
2019-11-06 10:51:08 ending get_db_url ; time: 0:00:00.000057
2019-11-06 10:52:24 starting check_df
2019-11-06 10:52:24 starting frame_splain
DATAFRAME SHAPE:
(77381, 11) 

DATAFRAME INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 11 columns):
parcelid                        77381 non-null int64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
latitude                        77381 non-null float64
longitude                       77381 non-null float64
propertylandusetypeid           77381 non-null float64
propertylandusedesc             77381 non-null object
logerror                        77381 non-null float64
transactions                    773

2. **Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.).**

In [6]:
df = dfo.df

In [7]:
frame_splain(dfo.df, title='Zillow Data', splain=True)

2019-11-06 10:52:24 starting frame_splain
ZILLOW DATA SHAPE:
(77381, 11) 

ZILLOW DATA INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 11 columns):
parcelid                        77381 non-null int64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
latitude                        77381 non-null float64
longitude                       77381 non-null float64
propertylandusetypeid           77381 non-null float64
propertylandusedesc             77381 non-null object
logerror                        77381 non-null float64
transactions                    77381 non-null int64
dtypes: float64(8), int64(2), object(1)
memory usage: 6.5+ MB
None 

ZILLOW DATA DESCRIPTION:
                                count          mean           std  \
parcelid                      77381

In [8]:
@timeifdebug
def convert_to_dates(df, cols=[], format='%Y-%m-%d', errors='coerce', **kwargs):
    '''
    convert_to_dates(
        df, 
        cols, 
        format='%y-%m-%d', 
        errors='coerce', 
        **kwargs
    )
    RETURNS df with columns updated to dates
    '''
    convertcols = [col for col in cols if col in df.columns]
    for col in convertcols:
        df[col] = pd.to_datetime(df[col], errors=errors, format=format)
    return df

In [9]:
df = convert_to_dates(df, cols=['transactiondate'])

2019-11-06 10:52:24 starting convert_to_dates
2019-11-06 10:52:24 ending convert_to_dates ; time: 0:00:00.000685


In [10]:
dfo.df.head(15)

,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,latitude,longitude,propertylandusetypeid,propertylandusedesc,logerror,transactions
0,14297519,3.5,4.0,3.5,3100.0,33634931.0,-117869207.0,261.0,Single Family Residential,0.025595,1
1,17052889,1.0,2.0,1.0,1465.0,34449266.0,-119281531.0,261.0,Single Family Residential,0.055619,1
2,14186244,2.0,3.0,2.0,1243.0,33886168.0,-117823170.0,261.0,Single Family Residential,0.005383,1
3,12177905,3.0,4.0,3.0,2376.0,34245180.0,-118240722.0,261.0,Single Family Residential,-0.103410,1
4,10887214,3.0,3.0,3.0,1312.0,34185120.0,-118414640.0,266.0,Condominium,0.006940,1
5,17143294,2.0,3.0,2.0,1492.0,34230044.0,-118993991.0,266.0,Condominium,-0.020526,1
6,12095076,3.0,4.0,3.0,2962.0,34145202.0,-118179824.0,261.0,Single Family Residential,-0.001011,1
7,12069064,1.0,2.0,1.0,738.0,34149214.0,-118239357.0,261.0,Single Family Residential,0.101723,1
8,12790562,3.0,4.0,3.0,3039.0,33960230.0,-118006914.0,261.0,Single Family Residential,-0.040966,1
9,11542646,3.0,2.0,3.0,1290.0,33998800.0,-118416000.0,266.0,Condominium,-0.036763,1


In [11]:
frame_splain(dfo.df, splain=True)

2019-11-06 10:52:24 starting frame_splain
DATAFRAME SHAPE:
(77381, 11) 

DATAFRAME INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 11 columns):
parcelid                        77381 non-null int64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
latitude                        77381 non-null float64
longitude                       77381 non-null float64
propertylandusetypeid           77381 non-null float64
propertylandusedesc             77381 non-null object
logerror                        77381 non-null float64
transactions                    77381 non-null int64
dtypes: float64(8), int64(2), object(1)
memory usage: 6.5+ MB
None 

DATAFRAME DESCRIPTION:
                                count          mean           std  \
parcelid                      77381.0  1.

3: **Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.**

||num_rows_missing|pct_rows_missing|
|:---|---:|---:|
|parcelid|0|0.000000|
|airconditioningtypeid|29041|0.535486|
|architecturalstyletypeid|54232|0.999982|


In [12]:
@timeifdebug
def get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='',
        limit_to_max=False,
        **kwargs
        ):
    '''
    get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='', 
        limit_to_max=False,
        **kwargs
        )
    RETURNS summary dataframe

    Receives dataframe as input, examines all columns defined as objects, and
    returns a summary report with column name as its index.

    Report shows the following information about the columns of the dataframe:
        cols: column name
        dtypes: data type
        num_rows_values: number of rows with non-null values 
        num_rows_missing: number of rows with missing values
        pct_rows_missing: percentage of rows with missing values
        num_uniques: number of unique values 
        unique_values: list of unique values if the unique value count is less than 
            or equal to max_uniques.
        
    If the input dataframe contains the target column, enter that name as the 
    target_col argument and it will be removed from the analysis.
    
    If limit_to_max is True (default), the resulting dataframe will only show
    columns with no more unique values than specified in max_uniques.
    
    NOTE: This function can be used to identify and then re-type catergorical
    columns. Run the function with the following parameters, change the data 
    types of know categorical values to 'category', then re-run this function, 
    gradually increasing the max_uniques value until the desired limit for 
    categoricals has been met.
    
    get_column_values_stats(
        df, 
        get_cols=df.columns[df.dtypes!='category'],
        max_uniques=5,
        limit_to_max=True
        )
    '''
    # get rowcount
    num_recs = len(df)
    
    # if no columns presented, get all columns from df
    use_cols = df.columns if len(get_cols) == 0 else get_cols
    # ensure use_cols are actually present in df
    cols = [col for col in use_cols if col in df.columns]
    
    # make df for all columns, exclude target column if passed
    df_cols = pd.DataFrame(cols, columns=['cols'])
    df_cols = df_cols[df_cols.cols != target_col]
    
    # get statistics on all columns
    df_cols['dtype'] = df_cols.cols.apply(lambda x: df[x].dtype)
    df_cols['num_rows_values'] = df_cols.cols.apply(lambda x: df[x].count())
    df_cols['num_rows_missing'] = num_recs - df_cols.num_rows_values
    df_cols['pct_rows_missing'] = df_cols.num_rows_missing / num_recs
    df_cols['num_uniques'] = df_cols.cols.apply(lambda x: df[x].nunique())
    
    # get unique valeues for columns within unique value limits
    df_cats = df_cols[df_cols.num_uniques <= max_uniques]
    df_cats['unique_values'] = df_cats.cols.apply(lambda x: df[x].unique())
    
    # merge and set index
    join_type = 'inner' if limit_to_max else 'left'
    df_cols = df_cols.join(df_cats.unique_values, how=join_type)
    return df_cols.set_index('cols')

**Find string categoricals**

In [13]:
df_strings = get_column_values_stats(
    df, 
    get_cols=df.columns[df.dtypes=='object'],
    max_uniques=20000,
    limit_to_max=True
)
df_strings

2019-11-06 10:52:24 starting get_column_values_stats
2019-11-06 10:52:24 ending get_column_values_stats ; time: 0:00:00.070400


,dtype,num_rows_values,num_rows_missing,pct_rows_missing,num_uniques,unique_values
cols,,,,,,
propertylandusedesc,object,77381,0,0.0,13,"[Single Family Residential, Condominium, Duple..."


In [14]:
df_strings.index

Index(['propertylandusedesc'], dtype='object', name='cols')

In [15]:
string_categoricals = [
    'airconditioningdesc', 
    'architecturalstyledesc', 
    'buildingclassdesc', 
    'county', 
    'state', 
    'heatingorsystemdesc', 
    'propertycountylandusecode', 
    'propertylandusedesc', 
    'propertyzoningdesc', 
    'storydesc', 
    'taxdelinquencyflag', 
    'typeconstructiondesc'
]

In [16]:
df = prep.retype_cols(df, cols=string_categoricals, to_dtype='category')
df.info()

2019-11-06 10:52:25 starting retype_cols
2019-11-06 10:52:25 ending retype_cols ; time: 0:00:00.051049
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 11 columns):
parcelid                        77381 non-null int64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
latitude                        77381 non-null float64
longitude                       77381 non-null float64
propertylandusetypeid           77381 non-null float64
propertylandusedesc             77381 non-null category
logerror                        77381 non-null float64
transactions                    77381 non-null int64
dtypes: category(1), float64(8), int64(2)
memory usage: 6.0 MB


**Find Numeric Categoricals**

In [17]:
df_cat_check = get_column_values_stats(
    df, 
    get_cols=df.columns[df.dtypes!='category'],
    max_uniques=750000,
    limit_to_max=True
)
df_cat_check

2019-11-06 10:52:25 starting get_column_values_stats
2019-11-06 10:52:25 ending get_column_values_stats ; time: 0:00:00.222857


,dtype,num_rows_values,num_rows_missing,pct_rows_missing,num_uniques,unique_values
cols,,,,,,
parcelid,int64,77381,0,0.000000,77381,"[14297519, 17052889, 14186244, 12177905, 10887..."
bathroomcnt,float64,77381,0,0.000000,22,"[3.5, 1.0, 2.0, 3.0, 0.0, 2.5, 4.5, 4.0, 5.0, ..."
bedroomcnt,float64,77381,0,0.000000,16,"[4.0, 2.0, 3.0, 0.0, 6.0, 5.0, 1.0, 7.0, 8.0, ..."
calculatedbathnbr,float64,76772,609,0.007870,21,"[3.5, 1.0, 2.0, 3.0, nan, 2.5, 4.5, 4.0, 5.0, ..."
calculatedfinishedsquarefeet,float64,77185,196,0.002533,4972,"[3100.0, 1465.0, 1243.0, 2376.0, 1312.0, 1492...."
latitude,float64,77381,0,0.000000,64038,"[33634931.0, 34449266.0, 33886168.0, 34245180...."
longitude,float64,77381,0,0.000000,62461,"[-117869207.0, -119281531.0, -117823170.0, -11..."
propertylandusetypeid,float64,77381,0,0.000000,13,"[261.0, 266.0, 246.0, 260.0, 269.0, 247.0, 248..."
logerror,float64,77381,0,0.000000,76837,"[0.025594901758400002, 0.0556190874015, 0.0053..."


In [18]:
df_cat_check.index

Index(['parcelid', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'latitude', 'longitude',
       'propertylandusetypeid', 'logerror', 'transactions'],
      dtype='object', name='cols')

In [19]:
numeric_categoricals = [
    'airconditioningtypeid', 
    'architecturalstyletypeid', 
    'buildingclasstypeid', 
    'buildingqualitytypeid',
    'decktypeid', 
    'fips', 
    'hashottuborspa',
    'heatingorsystemtypeid',
    'pooltypeid10', 
    'pooltypeid2', 
    'pooltypeid7', 
    'propertylandusetypeid',
    'regionidcounty', 
    'regionidcity',
    'regionidneighborhood',
    'regionidzip',
    'rawcensustractandblock',
    'censustractandblock',
    'storytypeid', 
    'threequarterbathnbr', 
    'numberofstories', 
    'fireplaceflag', 
    'assessmentyear', 
    'typeconstructiontypeid', 
    'transactions'
]

In [20]:
df = prep.retype_cols(df, cols=numeric_categoricals, to_dtype='category')
df.info()

2019-11-06 10:52:25 starting retype_cols
2019-11-06 10:52:25 ending retype_cols ; time: 0:00:00.019350
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 11 columns):
parcelid                        77381 non-null int64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
latitude                        77381 non-null float64
longitude                       77381 non-null float64
propertylandusetypeid           77381 non-null category
propertylandusedesc             77381 non-null category
logerror                        77381 non-null float64
transactions                    77381 non-null category
dtypes: category(3), float64(7), int64(1)
memory usage: 4.9 MB


4. **Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.**

num_cols_missing|pct_cols_missing|num_rows
:---|---:|---:
23|38.333|108
24|40.000|123
25|41.667|5280

In [21]:
def nulls_by_row(df):
    total_cols = df.shape[1]
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/ total_cols
    rows_missing = (
        pd.DataFrame(
        {
            'num_cols_missing': num_cols_missing, 
            'pct_cols_missing': pct_cols_missing
        }
        )
        .reset_index()
        .groupby(['num_cols_missing','pct_cols_missing'])
        .count()
        .rename(index=str, columns={'index': 'num_rows'})
        .reset_index()
    )
    return rows_missing 

In [22]:
nulls_by_row(df)

,num_cols_missing,pct_cols_missing,num_rows
0,0,0.0,76762
1,1,0.09090909090909091,433
2,2,0.18181818181818182,186


#### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2. Create a function that will drop rows or columns based on the percent of values that are missing: `handle_missing_values(df, prop_required_column, prop_required_row)`.

   - The input:
       - A dataframe
       - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
       - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
   - The output:
       - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
   - hint:
       - Look up the dropna documentation.
       - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
       - Make use of inplace, i.e. inplace=True/False.

In [23]:
def handle_missing_values(
    df, 
    prop_required_column, 
    prop_required_row
    ):
    '''
    handle_missing_values(
    df, 
    prop_required_column=.9, 
    prop_required_row=.9
    )
    '''
    

3. Decide how to handle the remaining missing values:
   - Fill with constant value.
   - Impute with mean, median, mode.
   - Drop row/column

In [24]:
df_sqft = df[['calculatedfinishedsquarefeet',
'finishedfloor1squarefeet',
'finishedsquarefeet6',
'finishedsquarefeet12',
'finishedsquarefeet13',
'finishedsquarefeet15',
'finishedsquarefeet50'
]]

KeyError: "['finishedsquarefeet13', 'finishedsquarefeet12', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'finishedfloor1squarefeet'] not in index"

#### `wrangle_zillow.py`

Functions of the work above needed to acquire and prepare a new sample of data.

In [ ]:
df_sqft[df_sqft.calculatedfinishedsquarefeet.isna() & (df_sqft.finishedsquarefeet50.isna()==False)]

In [ ]:
df_pool = df[['poolcnt','poolsizesum','pooltypeid2','pooltypeid7','pooltypeid10']]
df_pool[(df_pool.poolcnt.isna()==False) | (df_pool.pooltypeid10.isna()==False)]